In [2]:
%pip install ipynb

# Run the above line to get the preprocess() function from Preprocessing.ipynb

Note: you may need to restart the kernel to use updated packages.


In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from ipynb.fs.full.Preprocessing import preprocess

In [84]:
train_x, train_y, test_x, test_y = preprocess(10000)

train_y = train_y.reshape(-1, 1)
train_y = train_y.astype('int')

train_x = train_x.astype("float")
test_x = test_x.astype("float")

test_y = test_y.reshape(-1, 1)
test_y = test_y.astype('int')

In [85]:
#We have 12 features, add 1 column (as 1st row) to the train_x data 
#train_x = np.c_[np.ones( train_x.shape[0] ), train_x]
train_x.shape

(7948, 12)

In [86]:
def LogisticRegressionTrain(train_x, train_y):
    number_of_classes = 3
    number_of_features = train_x.shape[1]
    number_of_samples = train_x.shape[0]
    learning_rate = 0.5
    
    assert(number_of_features == 12)
    
    # One-hot encode the classes for training
    # E.g., if class array is [0, 0, 1, 0, 2]
    # Convert to [[1,0,0], [1,0,0], [0,1,0], [1,0,0],[0,0,2]]
    one_hot_encoded = []

    for arr in (train_y):
        if (arr == [0]):
            one_hot_encoded.append([1, 0, 0,])
        elif (arr == [1]):
            one_hot_encoded.append([0, 1, 0,])
        elif (arr == [2]):
            one_hot_encoded.append([0, 0, 1,])
        else:
            print("error")

    encoded_train_y = np.array(one_hot_encoded)

    # Initialize the weights
    weights = np.random.rand(number_of_features, number_of_classes)
    bias = np.random.randn(1, number_of_classes)
    
    iteration_count = 1000
    for i in range(1, iteration_count + 1):
        # Get the Log-odds (logit)
        logits = train_x @ weights + bias

        # Calculate the probability of y 
        # Pr(Y = 0, 1, or 2 | X = train_x)
        max_value = np.max(logits, axis =1, keepdims=True)
        numerator = np.exp(logits - max_value)
        denominator = np.sum(numerator, axis=-1, keepdims=True)
        y_probability = numerator / denominator
                
        # Calculate the gradient vectors 
        
        # Gradient of the weights
        dW = (1 / number_of_samples) * (train_x.T @ (y_probability - encoded_train_y))
        
        # Gradient of the bias
        db = (1 / number_of_samples) * np.sum((y_probability - encoded_train_y), axis=0, keepdims=True)
        
        # We use gradient descent to update the weights and bias
        weights -= (dW * learning_rate)
        bias -= (db * learning_rate)
        
    return weights, bias
        

    

In [87]:
def make_prediction(feature, weights, bias, threshold=0.5):
    logits = feature @ weights + bias

    max_value = np.max(logits, axis =1, keepdims=True)
    numerator = np.exp(logits - max_value)
    denominator = np.sum(numerator, axis=-1, keepdims=True)
    y_probability = numerator / denominator
    
    return np.argmax(y_probability, axis=1)

In [88]:
def calc_pred_accuracy(test_x, test_y, weights, bias):
    prediction = make_prediction(test_x, weights, bias)

    accurate_prediction_count = 0
    for i, pred in enumerate(prediction):
        if pred == test_y[i][0]:
            accurate_prediction_count += 1
            
    return accurate_prediction_count / test_y.shape[0]
        


In [89]:
weights, bias = LogisticRegressionTrain(train_x, train_y)

In [90]:
print(calc_pred_accuracy(test_x, test_y, weights, bias))

0.9155
